In [1]:
!pip install pyspark==3.3.1


     |████████████████████████████████| 281.4 MB 17 kB/s  eta 0:00:011  |                                | 573 kB 3.2 MB/s eta 0:01:27     |                                | 727 kB 3.2 MB/s eta 0:01:27     |▏                               | 1.7 MB 3.2 MB/s eta 0:01:27     |▍                               | 3.4 MB 3.2 MB/s eta 0:01:26     |▌                               | 4.7 MB 3.8 MB/s eta 0:01:13     |▋                               | 5.8 MB 3.8 MB/s eta 0:01:13     |▊                               | 5.9 MB 3.8 MB/s eta 0:01:13     |▉                               | 7.1 MB 3.8 MB/s eta 0:01:13     |▉                               | 7.5 MB 3.8 MB/s eta 0:01:12     |▉                               | 7.7 MB 2.3 MB/s eta 0:01:59     |▉                               | 7.8 MB 2.3 MB/s eta 0:01:59     |█▎                              | 11.7 MB 2.3 MB/s eta 0:01:57     |█▍                              | 12.2 MB 2.3 MB/s eta 0:01:57     |█▍                              | 12.5 MB 6.8 MB/s eta

In [1]:
from pyspark.sql import SparkSession


spark = SparkSession\
    .builder \
    .appName("transformacao Twiiter") \
    .getOrCreate()


your 131072x1 screen size is bogus. expect trouble


24/10/14 10:33:34 WARN Utils: Your hostname, AnselmoHome resolves to a loopback address: 127.0.1.1; using 172.17.64.178 instead (on interface eth0)
24/10/14 10:33:34 WARN Utils: Set SPARK_LOCAL_IP if you need to bind to another address


Setting default log level to "WARN".
To adjust logging level use sc.setLogLevel(newLevel). For SparkR, use setLogLevel(newLevel).


24/10/14 10:33:37 WARN NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable


In [2]:
df = spark.read.json("../../datalake/twitter_datascience/")


In [3]:
df.show()

+--------------------+--------------------+------------------+------------+
|                data|            includes|              meta|extract_data|
+--------------------+--------------------+------------------+------------+
|[{78, 43, 2024-10...|{[{2024-10-10T13:...|{1234567890abcdef}|  2024-10-10|
|[{92, 44, 2024-10...|{[{2024-10-10T17:...|              null|  2024-10-10|
|[{43, 5, 2024-10-...|{[{2024-10-12T19:...|{1234567890abcdef}|  2024-10-12|
|[{32, 13, 2024-10...|{[{2024-10-12T22:...|              null|  2024-10-12|
|[{79, 69, 2024-10...|{[{2024-10-11T08:...|              null|  2024-10-11|
|[{67, 45, 2024-10...|{[{2024-10-13T06:...|              null|  2024-10-13|
+--------------------+--------------------+------------------+------------+



In [4]:
df.printSchema()

root
 |-- data: array (nullable = true)
 |    |-- element: struct (containsNull = true)
 |    |    |-- author_id: string (nullable = true)
 |    |    |-- conversation_id: string (nullable = true)
 |    |    |-- created_at: string (nullable = true)
 |    |    |-- edit_history_tweet_ids: array (nullable = true)
 |    |    |    |-- element: long (containsNull = true)
 |    |    |-- id: string (nullable = true)
 |    |    |-- in_reply_to_user_id: string (nullable = true)
 |    |    |-- lang: string (nullable = true)
 |    |    |-- public_metrics: struct (nullable = true)
 |    |    |    |-- like_count: long (nullable = true)
 |    |    |    |-- quote_count: long (nullable = true)
 |    |    |    |-- reply_count: long (nullable = true)
 |    |    |    |-- retweet_count: long (nullable = true)
 |    |    |-- text: string (nullable = true)
 |-- includes: struct (nullable = true)
 |    |-- users: array (nullable = true)
 |    |    |-- element: struct (containsNull = true)
 |    |    |    |-- c

In [5]:
from pyspark.sql import functions as f

In [6]:
df.select(f.explode('data')).printSchema()

root
 |-- col: struct (nullable = true)
 |    |-- author_id: string (nullable = true)
 |    |-- conversation_id: string (nullable = true)
 |    |-- created_at: string (nullable = true)
 |    |-- edit_history_tweet_ids: array (nullable = true)
 |    |    |-- element: long (containsNull = true)
 |    |-- id: string (nullable = true)
 |    |-- in_reply_to_user_id: string (nullable = true)
 |    |-- lang: string (nullable = true)
 |    |-- public_metrics: struct (nullable = true)
 |    |    |-- like_count: long (nullable = true)
 |    |    |-- quote_count: long (nullable = true)
 |    |    |-- reply_count: long (nullable = true)
 |    |    |-- retweet_count: long (nullable = true)
 |    |-- text: string (nullable = true)



In [9]:
df.select(f.explode('data')).show(truncate=False)

+----------------------------------------------------------------------------------------------------------------------------------------------------------------+
|col                                                                                                                                                             |
+----------------------------------------------------------------------------------------------------------------------------------------------------------------+
|{78, 43, 2024-10-10T15:46:38.284539+0000, [35], 74, 22, en, {22, 40, 44, 53}, Outro tweet fictício relacionado a datascience}                                   |
|{57, 37, 2024-10-10T12:27:29.276520+0000, [33], 80, 0, en, {82, 78, 49, 69}, Tweet fictício criado usando inteligência artificial para falar sobre datascience} |
|{67, 72, 2024-10-10T04:14:29.654440+0000, [10], 76, 19, en, {82, 42, 57, 71}, Um terceiro tweet fictício sobre datascience}                                     |
|{89, 98, 2024-10-10T1

In [12]:
df.select(f.explode("data").alias("tweets"))\
    .select("tweets.author_id", "tweets.conversation_id",\
             "tweets.created_at", "tweets.id",\
             "tweets.public_metrics.*", "tweets.text")\
    .printSchema()

root
 |-- author_id: string (nullable = true)
 |-- conversation_id: string (nullable = true)
 |-- created_at: string (nullable = true)
 |-- id: string (nullable = true)
 |-- like_count: long (nullable = true)
 |-- quote_count: long (nullable = true)
 |-- reply_count: long (nullable = true)
 |-- retweet_count: long (nullable = true)
 |-- text: string (nullable = true)



In [13]:
df_tweetts = df.select(f.explode("data").alias("tweets"))\
    .select("tweets.author_id", "tweets.conversation_id",\
             "tweets.created_at", "tweets.id",\
             "tweets.public_metrics.*", "tweets.text")


In [14]:
df_tweetts.show(truncate=False)

+---------+---------------+-------------------------------+---+----------+-----------+-----------+-------------+---------------------------------------------------------------------------------+
|author_id|conversation_id|created_at                     |id |like_count|quote_count|reply_count|retweet_count|text                                                                             |
+---------+---------------+-------------------------------+---+----------+-----------+-----------+-------------+---------------------------------------------------------------------------------+
|78       |43             |2024-10-10T15:46:38.284539+0000|74 |22        |40         |44         |53           |Outro tweet fictício relacionado a datascience                                   |
|57       |37             |2024-10-10T12:27:29.276520+0000|80 |82        |78         |49         |69           |Tweet fictício criado usando inteligência artificial para falar sobre datascience|
|67       |72            

In [15]:
df.select(f.explode("includes.users"))

DataFrame[col: struct<created_at:string,id:string,name:string,username:string>]

In [16]:
df.select(f.explode("includes.users").alias("users"))

DataFrame[users: struct<created_at:string,id:string,name:string,username:string>]

In [17]:
df.select(f.explode("includes.users").alias("users")).select("users.*").printSchema()

root
 |-- created_at: string (nullable = true)
 |-- id: string (nullable = true)
 |-- name: string (nullable = true)
 |-- username: string (nullable = true)



In [18]:
user_df = df.select(f.explode("includes.users").alias("users")).select("users.*")

In [19]:
user_df.show(truncate=False)

+-------------------------------+---+-------+--------+
|created_at                     |id |name   |username|
+-------------------------------+---+-------+--------+
|2024-10-10T13:57:53.454099+0000|48 |User 1 |user1   |
|2024-10-10T02:35:30.581705+0000|48 |User 2 |user2   |
|2024-10-10T10:50:25.168466+0000|7  |User 3 |user3   |
|2024-10-10T21:04:12.049167+0000|21 |User 4 |user4   |
|2024-10-10T22:38:33.465490+0000|34 |User 5 |user5   |
|2024-10-10T02:19:48.335844+0000|16 |User 6 |user6   |
|2024-10-10T22:27:29.109963+0000|43 |User 7 |user7   |
|2024-10-10T19:40:47.255878+0000|33 |User 8 |user8   |
|2024-10-10T14:29:21.779256+0000|61 |User 9 |user9   |
|2024-10-10T15:34:39.519399+0000|26 |User 10|user10  |
|2024-10-10T17:30:06.239821+0000|59 |User 1 |user1   |
|2024-10-10T01:50:26.437753+0000|87 |User 2 |user2   |
|2024-10-10T02:35:03.638452+0000|33 |User 3 |user3   |
|2024-10-10T08:41:38.564309+0000|17 |User 4 |user4   |
|2024-10-10T09:29:37.135000+0000|42 |User 5 |user5   |
|2024-10-1

In [20]:
df_tweetts.coalesce(1).write.mode("overwrite").json('output/tweet')

In [ ]:
user_df.coalesce(1).write.mode("overwrite").json('output/user')